In [191]:
# Importing required packages
import pandas as pd
import numpy as np
from google.colab import drive
import nltk
from nltk import word_tokenize, FreqDist  
import sys
import io
import glob, os
import re
import codecs
import warnings
warnings.filterwarnings('ignore')

In [192]:
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [193]:
pip install pdfminer

In [194]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams

In [195]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [196]:
# Adopted code from https://github.com/felipemontano10/NLP-Causality-Extraction
#List of pdf documents
pdf = []
for file in glob.glob("/content/drive/My Drive/NLP/PDF/Doc PDFs/*.pdf"):
    pdf.append(file)
print(len(pdf))

# #Function to parse pdf files into text files 
def pdfparser(data):
    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data_out =  retstr.getvalue()
    b = data[:-4]+'.txt'
    f = open(b, 'w', encoding="utf-8")
    f.write(data_out)
    f.close
  
# Get the .txt files
for i in pdf:
    pdfparser(i)

In [197]:
## Set drive_path
drive_path = '/content/drive/My Drive/NLP/PDF/Doc PDFs/'
def load_in_data(drive_path):  

    os.chdir(drive_path)
    
    ## List the file names in the folder
    files = [x for x in os.listdir() if '.txt' in x]
    
    files.sort()
    
    text_data = ['']*len(files)
    
    i = 0
    for j in files:
        with open(j, encoding='utf-8') as file:
            text_data[i] = file.read()
        i = i+1
    
    ## Create a dict that matches file names to their text counterparts
    
    text_files_dict = {files[i]: text_data[i] for i in range(len(files))} 
    
    return files, text_data, text_files_dict

files, text_data, text_files_dict = load_in_data(drive_path)
print(files)

['a94amj.txt', 'adg08hrm.txt', 'afglmm10hrm.txt', 'am79amj.txt', 'amr.2019.0470.txt', 'ap81asq.txt', 'ar78amj.txt', 'atc05ijhrm.txt', 'ay14ijbs.txt', 'azw06meq.txt', 'b04ilr.txt', 'b99ilr.txt', 'bb95jibs.txt', 'bc11amj.txt', 'bc15apjm.txt', 'bgd05jom.txt', 'bl00aeam.txt', 'bl00amj.txt', 'bm05jbe.txt', 'bm72rm.txt', 'bmp09bjm.txt', 'bnk00jbr.txt', 'bp04beer.txt', 'bp06jms.txt', 'bp94amj.txt', 'breb08jom.txt', 'brm11jbe.txt', 'bs05jams.txt', 'bss03amj.txt', 'bw04jbr.txt', 'ces06ee.txt', 'cg01aaaj.txt', 'cg05smj.txt', 'chl08hrm.txt', 'chl09ijhrm.txt', 'ck99smj.txt', 'ckc10ajm.txt', 'cl09etp.txt', 'cl10pp.txt', 'clrv08aos.txt', 'cls06jbe.txt', 'clv11ijhrm.txt', 'cm80ar.txt', 'cs06amj.txt', 'ct05crr.txt', 'ctsm08jem.txt', 'cw09smj.txt', 'cww15ijpdlm.txt', 'cys14jcp.txt', 'd00jom.txt', 'd03jom.txt', 'dc05jms.txt', 'dd96amj.txt', 'dhy00ms.txt', 'dj01amj.txt', 'dtba07jap.txt', 'ekks05smj.txt', 'fasttext_dataset.txt', 'fcwz15jcp.txt', 'fo05jom.txt', 'fs90amj.txt', 'g03ijhrm.txt', 'gac11ijtr.txt

In [198]:
# sample text extracted from the pdf files
text_files_dict['a94amj.txt']

' \n\n \n\n \n\n \n\nEffects of Human Resource Systems on Manufacturing Performance and Turnover\nAuthor(s): Jeffrey B. Arthur\nSource: The Academy of Management Journal, Vol. 37, No. 3 (Jun., 1994), pp. 670-687\nPublished by: Academy of Management\nStable URL: https://www.jstor.org/stable/256705\nAccessed: 01-02-2019 20:40 UTC\n\nJSTOR is a not-for-profit service that helps scholars, researchers, and students discover, use, and build upon a wide\n\nrange of content in a trusted digital archive. We use information technology and tools to increase productivity and\n\nfacilitate new forms of scholarship. For more information about JSTOR, please contact support@jstor.org.\n\nYour use of the JSTOR archive indicates your acceptance of the Terms & Conditions of Use, available at\n\nhttps://about.jstor.org/terms\n\nAcademy of Management is collaborating with JSTOR to digitize, preserve and extend access\nto The Academy of Management Journal\n\nThis content downloaded from 152.15.47.133 on Fri

In [199]:
def clean_and_convert_to_sentence(text_file):
    
    j = text_file
    ## First, remove everything post the word REFERENCES or BIBLIOGRAPHY.
    ## Attach \n in case it appears in text 
    seps = ['REFERENCES\n','References\n','BIBLIOGRAPHY\n',
            'REFERENCES \n','References \n','BIBLIOGRAPHY \n']
    for sep in seps:
        if sep in j:
            j = j.replace(sep,sep.replace(' ',''))
            j = j.rsplit(sep, 1)[0]
    
    j = j.splitlines()
    
    ## Drop any line that is just a number or number and symbols
    j3 = [i for i in j if re.search('[a-zA-Z]', i.strip())]
    ## Drop any line if it is just a single character
    j3 = [i for i in j3 if len(i)>1]
    
    ## Drop any line that BEGINS with a month
    months = ['January','February','March','April','May','June',
              'July','August','September','October','November','December']
    
    months = [i.upper() for i in months]
    
    j3 = [i for i in j3 if i.split()[0].upper() not in months]
    
    
    ## Check for lines that end in a -
    
    for k in j3:
        if k.strip().endswith('-'):
            num = j3.index(k)
            # the end of the word is at the start of next line
            end = j3[num+1].split()[0]
            # we remove the - and append the end of the word
            j3[num] = k[:-1] + end
            # following space from the next line
            j3[num+1] = j3[num+1][len(end)+1:]

    ## Remove lines that contain "This content downloaded" and other download things
    
    for string in ['This content downloaded', 'downloaded', 'http','jstor','DOI','doi']:
        j4 = [x for x in j3 if string not in x]
  
    
    ## Remove lines IP addresses 
    regex = re.compile(r'(?:[\d]{1,3})\.(?:[\d]{1,3})\.(?:[\d]{1,3})\.(?:[\d]{1,3})')
    j5 = [i for i in j4 if not regex.search(i)]
    
    
    
    ## Remove everything in parentheses??
    ## Also need to worry about parentheses splitting over lines. 
    ## This bit of code inserts a linesplit placeholder and merges the text, 
    ## and then removes anything within parentheses
    j6 = j5
    for k in j6:
        j6[j6.index(k)] = k + " {LINESPLIT}"
    
    j6 = ''.join(i for i in j6)
    regex = r'\(.*?\)'
    j6 = re.sub(regex, '', j6)
    j6 = j6.split('{LINESPLIT}')
    j6 = [i for i in j6 if not i == '' ]
    j6 = [i for i in j6 if re.search('[a-zA-Z]', i.strip())]

    j7 = ''.join(i+' ' for i in j6)
    j7 = sent_tokenize(j7)
    
    ## replace double-spaces 
    for k in j7:
        j7[j7.index(k)] = k.replace('  ',' ')
    
    j7 = ''.join(i+' ' for i in j7)
    
    ## Replace cases of h1. or Hypothesis 1. with Hypo 1:
    ## Add a <split> signifier, so that when we extract hypotheses later
    ## we can split new lines based on <split>
    for num in np.arange(100):
        for regex in [r'h{}[a-zA-Z]\:'.format(num),r'H{}[a-zA-Z]\:'.format(num),
                      r'h{}[a-zA-Z]\.'.format(num),r'H{}[a-zA-Z]\.'.format(num),
                      r'h{}[a-zA-Z]'.format(num),r'H{}[a-zA-Z]'.format(num),
                      r'hypothesis {}[a-zA-Z]\:'.format(num),r'Hypothesis {}[a-zA-Z]\:'.format(num),
                      r'hypothesis {}[a-zA-Z]\.'.format(num),r'Hypothesis {}[a-zA-Z]\.'.format(num),
                      r'hypothesis {}[a-zA-Z]'.format(num),r'Hypothesis {}[a-zA-Z]'.format(num),
                      r'h{}\:'.format(num),r'H{}\:'.format(num),
                      r'h{}\.'.format(num),r'H{}\.'.format(num),
                      r'h{}'.format(num),r'H{}'.format(num),
                      r'hypothesis {}\:'.format(num),r'Hypothesis {}\:'.format(num),
                      r'hypothesis {}\.'.format(num),r'Hypothesis {}\.'.format(num),
                      r'hypothesis {}'.format(num),r'Hypothesis {}'.format(num),
                      r'p{}[a-zA-Z]\:'.format(num),r'P{}[a-zA-Z]\:'.format(num),
                      r'p{}[a-zA-Z]\.'.format(num),r'P{}[a-zA-Z]\.'.format(num),
                      r'p{}[a-zA-Z]'.format(num),r'P{}[a-zA-Z]'.format(num),
                      r'proposition {}[a-zA-Z]\:'.format(num),r'Proposition {}[a-zA-Z]\:'.format(num),
                      r'proposition {}[a-zA-Z]\.'.format(num),r'Proposition {}[a-zA-Z]\.'.format(num),
                      r'proposition {}[a-zA-Z]'.format(num),r'Proposition {}[a-zA-Z]'.format(num),
                      r'p{}\:'.format(num),r'P{}\:'.format(num),
                      r'p{}\.'.format(num),r'P{}\.'.format(num),
                      r'p{}'.format(num),r'P{}'.format(num),
                      r'proposition {}\:'.format(num),r'Proposition {}\:'.format(num),
                      r'proposition {}\.'.format(num),r'Proposition {}\.'.format(num),
                      r'proposition {}'.format(num),r'Proposition {}'.format(num),
                      ]:
            j7 =re.sub(regex,'<split>Hypo {}: '.format(num),j7)


    ## Just get rid of some annoying strings
    j7 = j7.replace(':  :',': ')
    ## Replace multiple white spaces with a single white space
    j7 = re.sub('\s+',' ',j7)
    ## Remove this string as it causes new sentences and funks up hypothesis extraction
    j7 = j7.replace(': .',':')

    return j7


In [200]:
text_cleaned_JB = ['']*len(text_data)
for i in np.arange(0,len(text_data)):
    print('{}% done '.format(round(i/len(text_data)*100)+1,1))
    text_cleaned_JB[i] = clean_and_convert_to_sentence(text_data[i])    

1% done 
2% done 
2% done 
3% done 
4% done 
5% done 
5% done 
6% done 
7% done 
7% done 
8% done 
9% done 
10% done 
10% done 
11% done 
12% done 
12% done 
13% done 
14% done 
15% done 
15% done 
16% done 
17% done 
17% done 
18% done 
19% done 
20% done 
20% done 
21% done 
22% done 
22% done 
23% done 
24% done 
25% done 
25% done 
26% done 
27% done 
27% done 
28% done 
29% done 
30% done 
30% done 
31% done 
32% done 
32% done 
33% done 
34% done 
35% done 
35% done 
36% done 
37% done 
37% done 
38% done 
39% done 
40% done 
40% done 
41% done 
42% done 
42% done 
43% done 
44% done 
45% done 
45% done 
46% done 
47% done 
47% done 
48% done 
49% done 
50% done 
50% done 
51% done 
52% done 
52% done 
53% done 
54% done 
55% done 
55% done 
56% done 
57% done 
57% done 
58% done 
59% done 
60% done 
60% done 
61% done 
62% done 
62% done 
63% done 
64% done 
65% done 
65% done 
66% done 
67% done 
67% done 
68% done 
69% done 
70% done 
70% done 
71% done 
72% done 
72% done 
73

In [201]:
## Save cleaned text
text_cleaned_dict_JB = {files[i]:text_cleaned_JB[i] for i in range(len(text_cleaned_JB))}
for key in text_cleaned_dict_JB.keys():
    text = text_cleaned_dict_JB[key]
    text = text.lower()
    text_file = codecs.open('Outputs/cleaned_text_data_jb/{}'.format(key), mode='a',encoding='utf-8')
    text_file.write(text)
    text_file.close()

In [265]:
def extract_hyps(files, text_cleaned_dict):
    
    ## Loop through the files and organize Hypothesis sentences into a dataframe
    i = 0
    for file in files:
        ## Pull text from the dictionary
        text = text_cleaned_dict[file]
        if len(text) == 0: continue

        text = text.lower()
        
        ## Use NLTK tokenize to parse text into sentences
        document = sent_tokenize(text)
        ## convert all text to lower
        document = [x.lower() for x in document if len(x.split())>5] 
        

        ## Split based on the occurrence of "hypothesis:" Want these on a new line,
        ## pfls is the document-specific list of hypothesis sentences
        pfls = ['']
        for pg in document:
            ## check if sentence contains a hypothesis-like sentence
            if 'hypo ' in pg:
                pfls.append(pg)
        pfls.remove(pfls[0])

        hyp_statements = ['']
        ## split the hyp_statements based in occurrence of hypo: (<split>)
        ## is the split identifier
        for hyp in pfls:
            hyp_out = hyp.split('<split>')
            hyp_out_new = [h for h in hyp_out if 'hypo' in h and len(h.split())>7]
            hyp_statements.append(hyp_out_new)
        hyp_statements.remove(hyp_statements[0])
        ## Flatten the list
        hyp_statements = [y for x in hyp_statements for y in x]
        # hyp_statements = [b for b in hyp_statements if b[-1] != ':']
        hyp_statements = [h for h in hyp_statements if len(h.split()) != 0]
        
        ## Just organize the data into a dataframe
        h_cols = ['h'+str(i) for i in range(len(hyp_statements))]
        file_cols = [file]*len(hyp_statements)
        ## Concatenate by document
        if i == 0:
            hypothesis = pd.DataFrame({'file_name': file_cols,
                                       'hypothesis_num': h_cols,
                                       'sentence': hyp_statements,
                                       'classification': 'Hypothesis'})
        else:
            hold = pd.DataFrame({'file_name': file_cols,
                                       'hypothesis_num': h_cols,
                                       'sentence': hyp_statements,
                                       'classification': 'Hypothesis'})
            hypothesis = pd.concat([hypothesis, hold],axis=0,ignore_index=True)
            
        i = i+1
    return hypothesis
    
hypothesis_sentences = extract_hyps(files, text_cleaned_dict_JB)

In [266]:
# Save the extracted hypothesis into csv file with  PaperID(filename), Hypothesis/Proposition #  and Hypothesis/Proposition Full Sentence columns
hypothesis_sentences_csv = hypothesis_sentences[['file_name','hypothesis_num', 'sentence']]
hypothesis_sentences_csv.columns = ['Paper ID', 'Hypothesis/Proposition #', 'Hypothesis/Proposition Full Sentence']
hypothesis_sentences_csv.to_csv('/content/drive/My Drive/NLP/PDF/DocText/hypothesis_sentences.csv')

In [203]:
# Hypothesis sentences used in Training Data
hypothesis_sentences = hypothesis_sentences[["sentence","classification"]]
hypothesis_sentences

,sentence,classification
0,hypo 1: plants with commitment human resource ...,Hypothesis
1,hypo 2: turnover will be higher in control hum...,Hypothesis
2,hypo 3: there will be a stronger negative rela...,Hypothesis
3,hypo 1: predicts that the presence of a commit...,Hypothesis
4,hypo 3: states that the negative relationship ...,Hypothesis
...,...,...
1153,hypo 6: ethical leadership moderates the indir...,Hypothesis
1154,"hypo 4: , under strong ethical leadership, csr...",Hypothesis
1155,hypo 6: by estimating the conditional indirect...,Hypothesis
1156,hypo 6: is thus supported figures 3 and 4 depi...,Hypothesis


In [204]:
def extract_non_hyps(files, text_cleaned_dict):
    
    ## Loop through the files and organize Non Hypothesis sentences into a dataframe
    i = 0
    for file in files:
        ## Pull text from the dictionary
        text = text_cleaned_dict[file]
        if len(text) == 0: continue

        text = text.lower()
        
        ## Use NLTK tokenize to parse text into sentences
        document = sent_tokenize(text)
        ## convert all text to lower
        document = [x.lower() for x in document if len(x.split())>5] 

        pfls = ['']
        for pg in document:
            ## Non hypothesis sentences for training data
            if not 'hypo ' in pg or not 'hypothesis' in pg or not 'hypotheses' in pg and len(pg)>10:
                pfls.append(pg)
        pfls.remove(pfls[0])
        
        ## Just organize the data into a dataframe
        file_cols = [file]*len(pfls)
        ## Concatenate by document
        if i == 0:
            non_hypothesis = pd.DataFrame({'file_name': file_cols,
                                       'sentence': pfls,
                                       'classification': 'Non_Hypothesis'})
        else:
            hold = pd.DataFrame({'file_name': file_cols,
                                       'sentence': pfls,
                                       'classification': 'Non_Hypothesis'})
            non_hypothesis = pd.concat([non_hypothesis, hold],axis=0,ignore_index=True)
            
        i = i+1
    return non_hypothesis
    
non_hypothesis = extract_non_hyps(files, text_cleaned_dict_JB)

In [241]:
# Non Hypothesis sentences used in Training Data
# Trimming Non Hypothesis sentences to balance the training data
non_hypothesis_sentences = non_hypothesis.sample(n=1142)
non_hypothesis_sentences= non_hypothesis_sentences[["sentence","classification"]]
non_hypothesis_sentences

,sentence,classification
16519,the residual scores ensured the independence o...,Non_Hypothesis
9665,this may explain why the relationship between ...,Non_Hypothesis
9926,all subscales displayed good internal reliabil...,Non_Hypothesis
6922,pay dispersion was measured with a gini coeffi...,Non_Hypothesis
12353,solutions to environmental downloaded by natio...,Non_Hypothesis
...,...,...
17962,they covered nine main areas of hrm: recruitme...,Non_Hypothesis
21384,"second, we veriﬁed empirically this typology, ...",Non_Hypothesis
13656,"thus, in this framework there are some attribu...",Non_Hypothesis
8182,human resource management doi: 10.1002/hrm str...,Non_Hypothesis


In [242]:
hypothesis_classification = hypothesis_sentences.append(non_hypothesis_sentences, ignore_index=True)
hypothesis_classification

,sentence,classification
0,hypo 1: plants with commitment human resource ...,Hypothesis
1,hypo 2: turnover will be higher in control hum...,Hypothesis
2,hypo 3: there will be a stronger negative rela...,Hypothesis
3,hypo 1: predicts that the presence of a commit...,Hypothesis
4,hypo 3: states that the negative relationship ...,Hypothesis
...,...,...
2295,they covered nine main areas of hrm: recruitme...,Non_Hypothesis
2296,"second, we veriﬁed empirically this typology, ...",Non_Hypothesis
2297,"thus, in this framework there are some attribu...",Non_Hypothesis
2298,human resource management doi: 10.1002/hrm str...,Non_Hypothesis


In [243]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [244]:
# Splitting the data into training and test data 
training_data, test_data = train_test_split(hypothesis_classification, test_size = 0.1, random_state = 0)

In [256]:
# Text Classification with Multinomial Naive Bayes
# combined all rows of the 'sentence' column
list_text = []
for words in training_data.sentence:
  list_text.append(words)

# Preprocessing data to feed into MultinomialNB Classifier
count_vect = CountVectorizer()
Text_train_counts = count_vect.fit_transform(list_text)
count_vect.vocabulary_.get(u'algorithm')
tf_transformer = TfidfTransformer(use_idf=False).fit(Text_train_counts)
X_train_tf = tf_transformer.transform(Text_train_counts)
tfidf_transformer = TfidfTransformer()
Text_train_tfidf = tfidf_transformer.fit_transform(Text_train_counts)

In [257]:
# Using MultinomialNB from sklearn.naive_bayes 
clf = MultinomialNB().fit(Text_train_tfidf, training_data.classification)

In [258]:
# Evaluation Model performance on the Test data
sentence_list = test_data['sentence'].to_list()
Text_new_counts = count_vect.transform(sentence_list)
Text_new_tfidf = tfidf_transformer.transform(Text_new_counts)
predicted = clf.predict(Text_new_tfidf)
classification_prediction_list =[]
for sentence, classification in zip(sentence_list, predicted):
  classification_prediction_list.append(classification)


In [259]:
# Naive_Bayes Model metrics
classification_list = test_data['classification'].to_list()
count = 0
for i in range(len(classification_list)):
  if classification_list[i]==classification_prediction_list[i]:
    count = count + 1
Accuracy = (count/len(classification_list))*100
print("Accuracy of MultinomialNB based text classification: {:.2f}".format(Accuracy))

Accuracy of MultinomialNB based text classification: 82.61


In [249]:
pip install fasttext

In [250]:
# Text Classification with fasttext

#Importing fasttext and other required packages
import fasttext
import csv

# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess
training_data.iloc[:, 0] = training_data.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
test_data.iloc[:, 0] = test_data.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
training_data.iloc[:, 1] = training_data.iloc[:, 1].apply(lambda x: '__label__' + x)
test_data.iloc[:, 1] = test_data.iloc[:, 1].apply(lambda x: '__label__' + x)

In [251]:
# Saving the CSV file as a text file to train and test the classifier
training_data[['classification', 'sentence']].to_csv('/content/drive/My Drive/NLP/PDF/DocText/training_data.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

test_data[['classification', 'sentence']].to_csv('/content/drive/My Drive/NLP/PDF/DocText/test_data.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")


# Training the fastText classifier
model = fasttext.train_supervised('/content/drive/My Drive/NLP/PDF/DocText/training_data.txt', wordNgrams = 2, lr = 0.3, dim = 200, loss="ns")

# Predicting on a single input
model.predict(test_data.iloc[2, 0])

(('__label__Non_Hypothesis',), array([0.99409896]))

In [254]:
# Evaluating performance on the entire test data
# Fasttext Model metrics
print(model.labels)
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
    print("Accuracy of Fasttext based text classification: {:.2f}".format(p*100))

print_results(*model.test('/content/drive/My Drive/NLP/PDF/DocText/test_data.txt'))

['__label__Non_Hypothesis', '__label__Hypothesis']
N	230
P@1	0.974
R@1	0.974
Accuracy of Fasttext based text classification: 97.4
